In [2]:
import sys,os,json,collections
import tifffile as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np

#ref1 https://github.com/darylclimb/image_affine_transform/blob/master/transformation.ipynb
#https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html

def rotateAndScale(img, scaleFactor = 0.5, InPlaneRot_Degree = 30):
    (oldY,oldX) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=InPlaneRot_Degree, scale=scaleFactor) #rotate about center of image.
    print(M)
    
    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(InPlaneRot_Degree)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    (tx,ty) = ((newX-oldX)/2,(newY-oldY)/2)
    M[0,2] += tx #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += ty

    rotatedImg = cv2.warpAffine(np.float32(img), M, (int(newX),int(newY)))
    return M, rotatedImg

In [3]:
img = cv2.imread('DB.tif').sum(-1)
img.shape

(960, 1280)

In [14]:
def rotateScaleTranslate(img, Translation = (1,1), scaleFactor = 0.5, InPlaneRot_Degree = 30):
    (oldY,oldX) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=InPlaneRot_Degree, scale=scaleFactor) #rotate about center of image.
    print(M)
    
    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(InPlaneRot_Degree)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    
    M[0,2] += Translation[0] #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += Translation[1]

    rotatedImg = cv2.warpAffine(np.float32(img), M, (int(newX),int(newY)))
    return M, rotatedImg

In [19]:
%matplotlib qt
Transmatrix, rot_image = rotateAndScale(img, scaleFactor = 2, InPlaneRot_Degree = 5)
fig, ax = plt.subplots(2,1)
ax[0].imshow(img)
ax[1].imshow(rot_image)
plt.show()

[[ 1.99238940e+00  1.74311485e-01 -7.18798727e+02]
 [-1.74311485e-01  1.99238940e+00 -3.64787559e+02]]


In [20]:
Transmatrix

array([[ 1.99238940e+00,  1.74311485e-01,  0.00000000e+00],
       [-1.74311485e-01,  1.99238940e+00,  2.23118701e+02]])

In [21]:
img[0,0]

132

In [24]:
Transmatrix@[0,0,1]

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)

In [25]:
def get_rotation(angle):
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0, 0, 1]
    ])
def get_translation(tx, ty):
    return np.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ])

def get_scale(s):
    return np.array([
        [s, 0, 0],
        [0, s, 0],
        [0, 0, 1]
    ])

R1 = get_rotation(135)
T1 = get_translation(-2, 2)
S1 = get_scale(2)

In [37]:
coords = [1,1,1]
M = S1@R1@T1

In [38]:
coords@M

array([ 2.22044605e-16, -2.82842712e+00, -4.65685425e+00])

In [39]:
s_c = coords@S1
rs_c = s_c@R1
trs_c = rs_c@T1
trs_c

array([ 2.22044605e-16, -2.82842712e+00, -4.65685425e+00])

In [40]:
M

array([[-1.41421356e+00, -1.41421356e+00, -4.44089210e-16],
       [ 1.41421356e+00, -1.41421356e+00, -5.65685425e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])